# Prdeicting Heart Failure Based on Clinical Data

This is in partial fulfillment of the course Statistical Modeling and simulation (CSMODEL). 

Submitted by: 
- ANG, Charlene
- CAI, Mark Jayson
- SANTOS, Carlo Luis
S15

Submitted to:

Mr. Thomas James Tiam-Lee 

September 22, 2020

## Heart Failure Prediction

https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

### Introduction

### Data Collection

## Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re 
from functools import reduce

### Loading the Dataset 

In [ ]:
df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
df.head()

### Dataset Information

In [ ]:
df.info()

In [ ]:
df.describe()

### Data Cleaning

#### Duplicates

In [ ]:
df.duplicated().sum()

#### Null values

In [ ]:
df.isna().sum().sort_values()

Based on the values presented above, there are no categorical data, no duplication of observations and no nullvalues present in the dataset, thus, the researchers will no longer perform data cleaning

### Exploratory Data Analysis Questions

**Question#1**: What are the average values of age that are most likely to contract heart failure?

First lets look at the age distribution of the patients

In [ ]:
df.hist("age", bins=30)
plt.show()

Let's now look at the average, median, and standard deviation age of the patients.

In [ ]:
df.agg({"age": ["mean", "median", "std", "count"]})

**Answer#1**: 60.83 is the average age in which patients contract a heart disease. 60 is the median age. And the standard deviation is 11.89.

**Question#2**: Is Age and Sex an indicator for Death Event?

Let's first look at the death event and the age of the patients.

In [ ]:
x = df.loc[df['DEATH_EVENT'] == 0, ['age']]
y = df.loc[df['DEATH_EVENT'] == 1, ['age']]

fig, ax = plt.subplots()
x.hist(alpha=1, ax=ax, label="Survived")
y.hist(alpha=0.5, ax=ax, label="Died")
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Histogram of the Survival State and age of the patients')
plt.legend(loc='upper right')
plt.show()

Let's now look at the death event and the sex of the patient.

In [ ]:
df.groupby(['DEATH_EVENT', 'sex']).size().unstack().plot(kind='bar', stacked='True')
plt.xlabel('Deceased')
plt.ylabel('Count')
plt.title('Bar plot of survival status and sex of the patients')
plt.legend(["Male", "Female"])
plt.show()

**Answer#2**:

**Question#3**:What is the distribution of patients in the following clinical features of the dataset?

Here we are going to look at the percentage of patients who have anemia, diabetes, and those who smoke.

#### Anaemia Percentage

In [ ]:
val = df['anaemia'].value_counts()
labels = ['Not Anemic', 'Anemic']
explode = (0.1, 0)
slices = plt.pie(val, labels = val, explode=explode, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Anemic percentage')
plt.legend(labels)
plt.show()

#### Diabetic Percentage

In [ ]:
val = df['diabetes'].value_counts()
labels = ['Not Diabetic', 'Diabetic']
explode = (0.1, 0)
slices = plt.pie(val, labels = val, explode=explode, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Diabetic percentage')
plt.legend(labels)
plt.show()

#### Smoking percentage

In [ ]:
val = df['smoking'].value_counts()
labels = ['Not Smoking', 'Smoking']
explode = (0.1, 0)
slices = plt.pie(val, labels = val, explode=explode, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Smoking percentage')
plt.legend(labels)
plt.show()

**Answer#3**:

**Question#4**: What is the correlation of each variable to the death event?
- Age
- Anaemia
- Creatinine Phosphokinase
- Diabetes
- Ejection fraction
- Hypertension
- Level of serum creatinine
- Level of serum sodium
- Platelets in the blood
- Smoking
- Sex


**Answer#4**:

## Research Questions

**Research Question #1** :Given the level of age, time, serum sodium, serum creatinine, and ejection fraction, what is the chance of heart failure?


**Research Question #2** : Find the confidence interval of the level of serum creatinine and ejection fraction for those who have heart failure death event between high blood and non high-blood?

## Insights and Conclusion